# Hate Speech Detector 2.0
---
**Best model errors analysis**

1. Load 1000 test examples and best hate speech detection model (**Simple Recurrent Neural Network**).
2. Predict hate speech for 1000 test tweets using best model. Utilize prediction probability.
3. For each of 7 hate speech classes perform:
    1. Most common words or word phrases analysis.
    2. Hashtags analysis.
    3. Tweet authors (sources) analysis.<br/><br/>
    Judging by:
    4. False negatives (1-st type errors)
    5. False positives (2-nd type errors)
    6. True negatives
    7. True positives

In [1]:
import numpy as np
import pandas as pd
import csv

from tqdm.notebook import tqdm

from src.dataframes.utils import classes, combine_row_wisely

from src.vectorizers.TextOwnTrainedFTVectorizer import TextOwnTrainedFTVectorizer
from src.nn.models.RecurrentNet import RecurrentNet
from src.classifiers.DLVectorClassifier import DLVectorClassifier
import torch

from src.error_analysis import (class_predictions, split_error_types,
                                most_common_word, most_common_phrase,
                                hashtags, usernames)
from src.constants import (LABELS,
                           COMBINED_TESTONLY_PATH, LEMMAS_TESTONLY_PATH)

In [2]:
CLF_CLASS = DLVectorClassifier
SHORT_NAME = 'RNN'
CLF_KWARGS = dict({
    'short_name': '5-1-0',
    'k_folds': 5,
    'vec_class': TextOwnTrainedFTVectorizer,
    'nn_class': RecurrentNet,
    'nn_type': 'recurrent_w2',
    'vec_params': dict({
        'model_type': 's',
        'short_name': 'super'
    }),
    'nn_params': dict({
        'n_layers': 5,
        'drop_prob': 0.1,
        'bidirectional': False
    })
})

## Error analysis plan
---
Based on [this Kaggle notebook](https://www.kaggle.com/jannen/model-error-analysis).

1. Enable the model returning prediction class probability values beside predicted classes.
2. Predict hate speech classes for 1000 annotated tweets (corpus only for tests).
3. For each of 7 hate speech classes:
    1. Utilize precise class belonging probability and calculate difference between ground truth.
    2. Utilize predicted classes (0 or 1) and split predictions into: **FN** (False Negatives), **FP** (False Positives), **TN** (True Negatives) and **TP** (True Positives).
    3. For each of above prediction types:
        1. Research empirically most common words and word phrases
        2. Research mean squared error values between different tweet sources (i.e. authors/users)
        3. Research which hashtags might have determined how big error values

## Data loading

### Classes

In [3]:
df_test_classes = pd.read_csv(COMBINED_TESTONLY_PATH)[LABELS]
df_test_classes = classes(df_test_classes, convert_null=True)
df_test_classes.head(2)

,wyzywanie,grożenie,wykluczanie,odczłowieczanie,poniżanie,stygmatyzacja,szantaż
0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0


### Lemmatized tweets, hashtags and sources (users)

In [4]:
df_test_lemmatized = pd.read_csv(LEMMAS_TESTONLY_PATH)[['lemmatized']]
df_test_lemmatized = combine_row_wisely([df_test_lemmatized,
                                         pd.read_csv(COMBINED_TESTONLY_PATH)[['username', 'hashtags']]])
df_test_lemmatized.head(2)

,lemmatized,username,hashtags
0,warto poznać śląsk i ślązak zanim zacząć się w...,arozenek,[]
1,jeszcze nie być całkiem bezkarny sąd jeszcze t...,koziolmk,[]


### Combined data

In [5]:
df_test_data = combine_row_wisely([df_test_lemmatized[['lemmatized']], df_test_classes])
df_test_data.head(2)

,lemmatized,wyzywanie,grożenie,wykluczanie,odczłowieczanie,poniżanie,stygmatyzacja,szantaż
0,warto poznać śląsk i ślązak zanim zacząć się w...,0,0,0,0,0,0,0
1,jeszcze nie być całkiem bezkarny sąd jeszcze t...,0,0,0,0,0,0,0


## Prediction

In [6]:
X_test_data = df_test_data.drop(LABELS, axis=1).values
y_test_data = df_test_data[LABELS].values

In [7]:
clf = DLVectorClassifier(**CLF_KWARGS)
clf.load()

y_pred, y_prob = clf.predict(X_test_data, return_probs=True)

In [8]:
y_pred.shape

(1022, 7)

In [9]:
y_prob.shape

(1022, 7)

In [10]:
y_test_data.shape

(1022, 7)

## Splitting into FN, FP, TN, TP for each hate speech class

### Wyzywanie

In [11]:
fn, fp, tn, tp = split_error_types(y_true=y_test_data, y_pred=y_pred, y_prob=y_prob, additional_data=df_test_lemmatized, cls='wyzywanie')

#### FN

In [12]:
fn.head(2)

,lemmatized,username,hashtags,ground truth,prediction,probability,error
10,złodziejstwo członek i sympatyk aws i po powin...,romualdmyga,['#wieszwiecej'],1,0,0.016349,0.983651
46,jebac pies w sąd wbijać,kebabzlodziej,[],1,0,0.007888,0.992112


In [13]:
most_common_word(fn['lemmatized'], min_count=4)

,word,count
5,sąd,35
120,sada,5
137,móc,4
185,pisowski,4
12,prokuratura,4


In [14]:
most_common_phrase(fn['lemmatized'], length=2).head()

,phrase,count
163,miać czekać,2
200,pisowski ścierwa,2
105,polski sąd,2
11,prokuratura sąd,2
90,sąd prokuratura,2


In [15]:
most_common_phrase(fn['lemmatized'], length=3).head()

,phrase,count
217,140 letni wlasciciel,1
223,24 duży gów,1
246,30 rok temu,1
131,administracyjny móc zinterpretować,1
143,andrzej miać czekać,1


In [16]:
most_common_phrase(fn['lemmatized'], length=4).head()

,phrase,count
216,30 rok temu kodziarstwo,1
114,administracyjny móc zinterpretować mielić,1
124,andrzej miać czekać robić,1
11,augiasz ogromny pole popis,1
3,aws zająć sąd nacisk,1


In [17]:
hashtags(fn)

/home/jakub/Dokumenty/Semestr 9.2/HSD2/src/error_analysis.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['hashtags'] = dff.apply(lambda x: list([ht[1:-1] for ht in x['hashtags'][1:-1].split(', ')]), axis=1)


,hashtag,error
1,#pinior,0.989123
2,#wieszwiecej,0.967570
4,#zmianacałodobowa,0.963120
3,#woronicza17,0.913872
0,#minęła20,0.845840


In [18]:
usernames(fn).head()

,username,error
18,nocri2,0.990245
21,pietro_v_volpi,0.990245
17,nagleociemnialy,0.989814
14,lemingopedia,0.989550
9,jacekban,0.989496


#### FP

In [19]:
fp.head(2)

,lemmatized,username,hashtags,ground truth,prediction,probability,error


In [20]:
most_common_word(fp['lemmatized'], min_count=4)

,word,count


In [21]:
most_common_phrase(fp['lemmatized'], length=2).head()

,phrase,count


In [22]:
most_common_phrase(fp['lemmatized'], length=3).head()

,phrase,count


In [23]:
most_common_phrase(fp['lemmatized'], length=4).head()

,phrase,count


In [24]:
hashtags(fp)

,hashtag,error


In [25]:
usernames(fp).head()

,username,error


#### TN

In [26]:
tn.head(2)

,lemmatized,username,hashtags,ground truth,prediction,probability,error
0,warto poznać śląsk i ślązak zanim zacząć się w...,arozenek,[],0,0,0.045615,0.045615
1,jeszcze nie być całkiem bezkarny sąd jeszcze t...,koziolmk,[],0,0,0.005065,0.005065


In [27]:
most_common_word(tn['lemmatized'], min_count=4)

,word,count
8,sąd,940
55,mieć,127
83,prokuratura,95
37,sada,92
155,polski,55
...,...,...
1695,złodziej,4
1257,złodziejstwo,4
214,łączewski,4
1652,życie,4


In [28]:
most_common_phrase(tn['lemmatized'], length=2).head()

,phrase,count
661,sąd prokuratura,28
172,polski sąd,27
502,prokuratura sąd,20
57,sąd mieć,18
687,niezawisły sąd,17


In [29]:
most_common_phrase(tn['lemmatized'], length=3).head()

,phrase,count
1200,komunizm nadal uważać,6
1201,nadal uważać sąd,6
5865,podporządkować siebie sąd,6
1202,uważać sąd chuligan,6
1199,walczyć komunizm nadal,6


In [30]:
most_common_phrase(tn['lemmatized'], length=4).head()

,phrase,count
1034,komunizm nadal uważać sąd,6
1035,nadal uważać sąd chuligan,6
1033,walczyć komunizm nadal uważać,6
5059,aparat bezpieczeństwo media publiczny,5
945,człowiek platfus móc spać,5


In [31]:
hashtags(tn)

/home/jakub/Dokumenty/Semestr 9.2/HSD2/src/error_analysis.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['hashtags'] = dff.apply(lambda x: list([ht[1:-1] for ht in x['hashtags'][1:-1].split(', ')]), axis=1)


,hashtag,error
66,#pomagamlemingom,0.060932
24,#francuskie,0.044671
48,#niewygodnefakty,0.044671
68,#postępowe,0.044671
94,#uśmiechnijsiefrancjo,0.044671
...,...,...
6,#audyty,0.000024
81,#smoleńsk,0.000024
20,#dziejesięhistoria,0.000024
82,#sn,0.000024


In [32]:
usernames(tn).head()

,username,error
237,iax2hffwvsuf5lg,0.170770
463,nova__mova,0.111902
695,zniedziolka,0.090674
456,najpolski,0.081338
11,_boreyko_,0.079652


#### TP

In [33]:
tp.head(2)

,lemmatized,username,hashtags,ground truth,prediction,probability,error


In [34]:
most_common_word(tp['lemmatized'], min_count=4)

,word,count


In [35]:
most_common_phrase(tp['lemmatized'], length=2).head()

,phrase,count


In [36]:
most_common_phrase(tp['lemmatized'], length=3).head()

,phrase,count


In [37]:
most_common_phrase(tp['lemmatized'], length=4).head()

,phrase,count


In [38]:
hashtags(tp)

,hashtag,error


In [39]:
usernames(tp).head()

,username,error


### Grożenie

In [40]:
fn, fp, tn, tp = split_error_types(y_true=y_test_data, y_pred=y_pred, y_prob=y_prob, additional_data=df_test_lemmatized, cls='grożenie')

#### FN

In [41]:
fn.head(2)

,lemmatized,username,hashtags,ground truth,prediction,probability,error
22,taaak rozpieprzyć sąd w rok powodzenie,barpa129i,[],1,0,0.079345,0.920655
40,tk musieć być normalny a sąd trzeba wyczyścić ...,wojciechhorwat,[],1,0,0.212776,0.787224


In [42]:
most_common_word(fn['lemmatized'], min_count=4)

,word,count
2,sąd,48
105,prokuratura,9
76,zlikwidować,6
22,polska,5
140,media,4
35,móc,4
141,ngo,4
165,państwo,4
50,polski,4
107,sada,4


In [43]:
most_common_phrase(fn['lemmatized'], length=2).head()

,phrase,count
161,sąd prokuratura,4
78,zlikwidować sąd,4
153,but podskoczyć,3
146,maziarski media,3
147,media sąd,3


In [44]:
most_common_phrase(fn['lemmatized'], length=3).head()

,phrase,count
129,maziarski media sąd,3
130,media sąd ngo,3
132,ngo se teatr,3
135,samorząd but podskoczyć,3
133,se teatr samorząd,3


In [45]:
most_common_phrase(fn['lemmatized'], length=4).head()

,phrase,count
112,maziarski media sąd ngo,3
113,media sąd ngo se,3
115,ngo se teatr samorząd,3
116,se teatr samorząd but,3
114,sąd ngo se teatr,3


In [46]:
hashtags(fn)

/home/jakub/Dokumenty/Semestr 9.2/HSD2/src/error_analysis.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['hashtags'] = dff.apply(lambda x: list([ht[1:-1] for ht in x['hashtags'][1:-1].split(', ')]), axis=1)


,hashtag,error
4,#takjest,0.976561
0,#maziarski,0.794853
3,#pinior,0.671832
5,#woronicza17,0.665583
1,#minęła20,0.529447
2,#państwowpaństwie,0.303153


In [47]:
usernames(fn).head()

,username,error
39,piotr_piotr_1,0.975806
29,kszalik,0.975606
3,alfabies,0.975535
26,krewetka02,0.975523
5,anitaschelde,0.975356


#### FP

In [48]:
fp.head(2)

,lemmatized,username,hashtags,ground truth,prediction,probability,error
195,przykład suwałk ukazywać prl korzenie sąd sąd ...,szlachciura,[],0,1,0.559278,0.559278
691,bo on gromadzić hak cel wymiana i handel wolny...,zniedziolka,[],0,1,0.534763,0.534763


In [49]:
most_common_word(fp['lemmatized'], min_count=4)

,word,count
5,sąd,6


In [50]:
most_common_phrase(fp['lemmatized'], length=2).head()

,phrase,count
38,agencja usługowy,1
23,areszt podstawa,1
32,biznesowy medialny,1
9,bolszewicki twór,1
14,cel wymiana,1


In [51]:
most_common_phrase(fp['lemmatized'], length=3).head()

,phrase,count
22,areszt podstawa reżim,1
30,biznesowy medialny rządzić,1
9,bolszewicki twór patologia,1
13,cel wymiana handel,1
19,demokracja pis sąd,1


In [52]:
most_common_phrase(fp['lemmatized'], length=4).head()

,phrase,count
28,biznesowy medialny rządzić s,1
9,bolszewicki twór patologia wykorzenić,1
12,cel wymiana handel wolny,1
18,demokracja pis sąd areszt,1
39,dokument fałszować sąd przyklepywać,1


In [53]:
hashtags(fp)

/home/jakub/Dokumenty/Semestr 9.2/HSD2/src/error_analysis.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['hashtags'] = dff.apply(lambda x: list([ht[1:-1] for ht in x['hashtags'][1:-1].split(', ')]), axis=1)


,hashtag,error


In [54]:
usernames(fp).head()

,username,error
2,szlachciura,0.312792
1,nova__mova,0.299557
3,zniedziolka,0.285972
0,andikg_62,0.259921


#### TN

In [55]:
tn.head(2)

,lemmatized,username,hashtags,ground truth,prediction,probability,error
0,warto poznać śląsk i ślązak zanim zacząć się w...,arozenek,[],0,0,0.079751,0.079751
1,jeszcze nie być całkiem bezkarny sąd jeszcze t...,koziolmk,[],0,0,0.012193,0.012193


In [56]:
most_common_word(tn['lemmatized'], min_count=4)

,word,count
8,sąd,914
55,mieć,126
37,sada,92
91,prokuratura,90
119,pis,54
...,...,...
108,zarzut,4
446,zasada,4
171,zbrodniarz,4
1583,zdanie,4


In [57]:
most_common_phrase(tn['lemmatized'], length=2).head()

,phrase,count
173,polski sąd,28
655,sąd prokuratura,26
318,prokuratura sąd,22
57,sąd mieć,18
672,niezawisły sąd,17


In [58]:
most_common_phrase(tn['lemmatized'], length=3).head()

,phrase,count
1169,komunizm nadal uważać,6
1170,nadal uważać sąd,6
5765,podporządkować siebie sąd,6
1171,uważać sąd chuligan,6
1168,walczyć komunizm nadal,6


In [59]:
most_common_phrase(tn['lemmatized'], length=4).head()

,phrase,count
1009,komunizm nadal uważać sąd,6
1010,nadal uważać sąd chuligan,6
1008,walczyć komunizm nadal uważać,6
4977,aparat bezpieczeństwo media publiczny,5
920,człowiek platfus móc spać,5


In [60]:
hashtags(tn)

/home/jakub/Dokumenty/Semestr 9.2/HSD2/src/error_analysis.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['hashtags'] = dff.apply(lambda x: list([ht[1:-1] for ht in x['hashtags'][1:-1].split(', ')]), axis=1)


,hashtag,error
64,#pomagamlemingom,0.248620
24,#francuskie,0.202888
47,#niewygodnefakty,0.202888
66,#postępowe,0.202888
91,#uśmiechnijsiefrancjo,0.202888
...,...,...
0,#3rp,0.000144
6,#audyty,0.000141
79,#smoleńsk,0.000141
20,#dziejesięhistoria,0.000140


In [61]:
usernames(tn).head()

,username,error
173,eric1dale,0.248620
223,holger_d69,0.244652
99,bogdan1409,0.234407
658,windswaysapp,0.202888
241,jacekkarnowski,0.195631


#### TP

In [62]:
tp.head(2)

,lemmatized,username,hashtags,ground truth,prediction,probability,error
25,to prosty bo sąd być nadal nie oczyścić,bolek_ma_tole,[],1,1,0.500101,0.499899
26,to prosty bo sąd być nadal nie oczyścić,bolek_ma_tole,[],1,1,0.500101,0.499899


In [63]:
most_common_word(tp['lemmatized'], min_count=4)

,word,count
1,sąd,7
3,oczyścić,4


In [64]:
most_common_phrase(tp['lemmatized'], length=2).head()

,phrase,count
2,nadal oczyścić,2
0,prosty sąd,2
1,sąd nadal,2
17,brukselski wykształciu,1
6,czas polikwidować,1


In [65]:
most_common_phrase(tp['lemmatized'], length=3).head()

,phrase,count
0,prosty sąd nadal,2
1,sąd nadal oczyścić,2
5,czas polikwidować mafia,1
23,dekomunizacja sąd działać,1
22,dokonać dekomunizacja sąd,1


In [66]:
most_common_phrase(tp['lemmatized'], length=4).head()

,phrase,count
0,prosty sąd nadal oczyścić,2
4,czas polikwidować mafia peowski,1
19,dekomunizacja sąd działać szkoda,1
18,dokonać dekomunizacja sąd działać,1
21,działać szkoda państwo polski,1


In [67]:
hashtags(tp)

/home/jakub/Dokumenty/Semestr 9.2/HSD2/src/error_analysis.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['hashtags'] = dff.apply(lambda x: list([ht[1:-1] for ht in x['hashtags'][1:-1].split(', ')]), axis=1)


,hashtag,error


In [68]:
usernames(tp).head()

,username,error
1,bolek_ma_tole,0.249899
5,szlachciura,0.241274
0,_boreyko_,0.224740
3,kgb08maria,0.193566
4,najpolski,0.188437


### Wykluczanie

In [69]:
fn, fp, tn, tp = split_error_types(y_true=y_test_data, y_pred=y_pred, y_prob=y_prob, additional_data=df_test_lemmatized, cls='wykluczanie')

#### FN

In [70]:
fn.head(2)

,lemmatized,username,hashtags,ground truth,prediction,probability,error
144,a ja tylko chciećbyć żeby sąd nie być od polit...,wojo1985,[],1,0,0.003777,0.996223
402,maziarski media sąd ngo se teatr samorząd pod ...,kom_obr_dem,[],1,0,0.069964,0.930036


In [71]:
most_common_word(fn['lemmatized'], min_count=4)

,word,count
1,sąd,5
7,media,4
8,ngo,4


In [72]:
most_common_phrase(fn['lemmatized'], length=2).head()

,phrase,count
12,but podskoczyć,3
5,maziarski media,3
6,media sąd,3
8,ngo se,3
11,samorząd but,3


In [73]:
most_common_phrase(fn['lemmatized'], length=3).head()

,phrase,count
4,maziarski media sąd,3
5,media sąd ngo,3
7,ngo se teatr,3
10,samorząd but podskoczyć,3
8,se teatr samorząd,3


In [74]:
most_common_phrase(fn['lemmatized'], length=4).head()

,phrase,count
3,maziarski media sąd ngo,3
4,media sąd ngo se,3
6,ngo se teatr samorząd,3
7,se teatr samorząd but,3
5,sąd ngo se teatr,3


In [75]:
hashtags(fn)

/home/jakub/Dokumenty/Semestr 9.2/HSD2/src/error_analysis.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['hashtags'] = dff.apply(lambda x: list([ht[1:-1] for ht in x['hashtags'][1:-1].split(', ')]), axis=1)


,hashtag,error
0,#maziarski,0.864968


In [76]:
usernames(fn).head()

,username,error
5,wojo1985,0.992461
2,kazimierziiiwlk,0.986470
0,gazeta_wyborcza,0.864968
3,kom_obr_dem,0.864968
4,pambocek,0.861737


#### FP

In [77]:
fp.head(2)

,lemmatized,username,hashtags,ground truth,prediction,probability,error


In [78]:
most_common_word(fp['lemmatized'], min_count=4)

,word,count


In [79]:
most_common_phrase(fp['lemmatized'], length=2).head()

,phrase,count


In [80]:
most_common_phrase(fp['lemmatized'], length=3).head()

,phrase,count


In [81]:
most_common_phrase(fp['lemmatized'], length=4).head()

,phrase,count


In [82]:
hashtags(fp)

,hashtag,error


In [83]:
usernames(fp).head()

,username,error


#### TN

In [84]:
tn.head(2)

,lemmatized,username,hashtags,ground truth,prediction,probability,error
0,warto poznać śląsk i ślązak zanim zacząć się w...,arozenek,[],0,0,0.050375,0.050375
1,jeszcze nie być całkiem bezkarny sąd jeszcze t...,koziolmk,[],0,0,0.003840,0.003840


In [85]:
most_common_word(tn['lemmatized'], min_count=4)

,word,count
8,sąd,970
55,mieć,129
91,prokuratura,98
37,sada,96
163,polski,58
...,...,...
651,zbrodnia,4
178,zbrodniarz,4
1654,zdanie,4
2458,zwykły,4


In [86]:
most_common_phrase(tn['lemmatized'], length=2).head()

,phrase,count
180,polski sąd,29
715,sąd prokuratura,29
342,prokuratura sąd,22
57,sąd mieć,18
741,niezawisły sąd,17


In [87]:
most_common_phrase(tn['lemmatized'], length=3).head()

,phrase,count
1254,komunizm nadal uważać,6
1255,nadal uważać sąd,6
6145,podporządkować siebie sąd,6
1256,uważać sąd chuligan,6
1253,walczyć komunizm nadal,6


In [88]:
most_common_phrase(tn['lemmatized'], length=4).head()

,phrase,count
1082,komunizm nadal uważać sąd,6
1083,nadal uważać sąd chuligan,6
1081,walczyć komunizm nadal uważać,6
5308,aparat bezpieczeństwo media publiczny,5
993,człowiek platfus móc spać,5


In [89]:
hashtags(tn)

/home/jakub/Dokumenty/Semestr 9.2/HSD2/src/error_analysis.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['hashtags'] = dff.apply(lambda x: list([ht[1:-1] for ht in x['hashtags'][1:-1].split(', ')]), axis=1)


,hashtag,error
65,#pomagamlemingom,0.043549
24,#francuskie,0.033980
47,#niewygodnefakty,0.033980
67,#postępowe,0.033980
93,#uśmiechnijsiefrancjo,0.033980
...,...,...
6,#audyty,0.000014
80,#smoleńsk,0.000014
20,#dziejesięhistoria,0.000014
81,#sn,0.000014


In [90]:
usernames(tn).head()

,username,error
242,iax2hffwvsuf5lg,0.113641
472,nova__mova,0.074098
710,zniedziolka,0.061189
464,najpolski,0.054568
12,_boreyko_,0.054535


#### TP

In [91]:
tp.head(2)

,lemmatized,username,hashtags,ground truth,prediction,probability,error


In [92]:
most_common_word(tp['lemmatized'], min_count=4)

,word,count


In [93]:
most_common_phrase(tp['lemmatized'], length=2).head()

,phrase,count


In [94]:
most_common_phrase(tp['lemmatized'], length=3).head()

,phrase,count


In [95]:
most_common_phrase(tp['lemmatized'], length=4).head()

,phrase,count


In [96]:
hashtags(tp)

,hashtag,error


In [97]:
usernames(tp).head()

,username,error


### Odczłowieczanie

In [98]:
fn, fp, tn, tp = split_error_types(y_true=y_test_data, y_pred=y_pred, y_prob=y_prob, additional_data=df_test_lemmatized, cls='odczłowieczanie')

#### FN

In [99]:
fn.head(2)

,lemmatized,username,hashtags,ground truth,prediction,probability,error
60,minąłać sąd do pełny dekomunizacja wraz z dzie...,szlachciura,['#minęła20'],1,0,0.058063,0.941937
95,w głęboki poważanie mieć prawo sąd tk kw ue on...,artur121977,[],1,0,0.002187,0.997813


In [100]:
most_common_word(fn['lemmatized'], min_count=4)

,word,count
1,sąd,9


In [101]:
most_common_phrase(fn['lemmatized'], length=2).head()

,phrase,count
41,pisowski ścierwa,2
13,sąd tk,2
20,banda odchyleniec,1
57,ból troszkę,1
37,ciągnąć rok,1


In [102]:
most_common_phrase(fn['lemmatized'], length=3).head()

,phrase,count
53,ból troszkę sąd,1
34,ciągnąć rok wykk,1
41,decyzja myśleć sąd,1
3,dekomunizacja wraz dziecko,1
49,drugi noc móc,1


In [103]:
most_common_phrase(fn['lemmatized'], length=4).head()

,phrase,count
48,ból troszkę sąd zabrać,1
31,ciągnąć rok wykk kruczkk,1
37,decyzja myśleć sąd przejąć,1
3,dekomunizacja wraz dziecko resortowy,1
44,drugi noc móc zasnąć,1


In [104]:
hashtags(fn)

/home/jakub/Dokumenty/Semestr 9.2/HSD2/src/error_analysis.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['hashtags'] = dff.apply(lambda x: list([ht[1:-1] for ht in x['hashtags'][1:-1].split(', ')]), axis=1)


,hashtag,error
0,#minęła20,0.887245


In [105]:
usernames(fn).head()

,username,error
6,tomciopaluch987,0.997576
4,michal_tadeusz,0.997447
0,artur121977,0.995631
1,gorszysortgd,0.992201
5,szlachciura,0.887245


#### FP

In [106]:
fp.head(2)

,lemmatized,username,hashtags,ground truth,prediction,probability,error


In [107]:
most_common_word(fp['lemmatized'], min_count=4)

,word,count


In [108]:
most_common_phrase(fp['lemmatized'], length=2).head()

,phrase,count


In [109]:
most_common_phrase(fp['lemmatized'], length=3).head()

,phrase,count


In [110]:
most_common_phrase(fp['lemmatized'], length=4).head()

,phrase,count


In [111]:
hashtags(fp)

,hashtag,error


In [112]:
usernames(fp).head()

,username,error


#### TN

In [113]:
tn.head(2)

,lemmatized,username,hashtags,ground truth,prediction,probability,error
0,warto poznać śląsk i ślązak zanim zacząć się w...,arozenek,[],0,0,0.007895,0.007895
1,jeszcze nie być całkiem bezkarny sąd jeszcze t...,koziolmk,[],0,0,0.001220,0.001220


In [114]:
most_common_word(tn['lemmatized'], min_count=4)

,word,count
8,sąd,966
55,mieć,128
91,prokuratura,98
37,sada,97
163,polski,58
...,...,...
178,zbrodniarz,4
1644,zdanie,4
2452,zwykły,4
222,łączewski,4


In [115]:
most_common_phrase(tn['lemmatized'], length=2).head()

,phrase,count
695,sąd prokuratura,30
180,polski sąd,29
342,prokuratura sąd,21
57,sąd mieć,18
721,niezawisły sąd,17


In [116]:
most_common_phrase(tn['lemmatized'], length=3).head()

,phrase,count
1239,komunizm nadal uważać,6
1240,nadal uważać sąd,6
6108,podporządkować siebie sąd,6
1241,uważać sąd chuligan,6
1238,walczyć komunizm nadal,6


In [117]:
most_common_phrase(tn['lemmatized'], length=4).head()

,phrase,count
1068,komunizm nadal uważać sąd,6
1069,nadal uważać sąd chuligan,6
1067,walczyć komunizm nadal uważać,6
5274,aparat bezpieczeństwo media publiczny,5
979,człowiek platfus móc spać,5


In [118]:
hashtags(tn)

/home/jakub/Dokumenty/Semestr 9.2/HSD2/src/error_analysis.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['hashtags'] = dff.apply(lambda x: list([ht[1:-1] for ht in x['hashtags'][1:-1].split(', ')]), axis=1)


,hashtag,error
66,#pomagamlemingom,0.030628
24,#francuskie,0.019319
48,#niewygodnefakty,0.019319
68,#postępowe,0.019319
94,#uśmiechnijsiefrancjo,0.019319
...,...,...
72,#prasówka,0.000001
25,#gabinetcieni,0.000001
54,#państwopis,0.000001
11,#bronka,0.000001


In [119]:
usernames(tn).head()

,username,error
240,iax2hffwvsuf5lg,0.130860
472,nova__mova,0.074182
464,najpolski,0.060007
712,zniedziolka,0.055433
12,_boreyko_,0.046450


#### TP

In [120]:
tp.head(2)

,lemmatized,username,hashtags,ground truth,prediction,probability,error


In [121]:
most_common_word(tp['lemmatized'], min_count=4)

,word,count


In [122]:
most_common_phrase(tp['lemmatized'], length=2).head()

,phrase,count


In [123]:
most_common_phrase(tp['lemmatized'], length=3).head()

,phrase,count


In [124]:
most_common_phrase(tp['lemmatized'], length=4).head()

,phrase,count


In [125]:
hashtags(tp)

,hashtag,error


In [126]:
usernames(tp).head()

,username,error


### Poniżanie

In [127]:
fn, fp, tn, tp = split_error_types(y_true=y_test_data, y_pred=y_pred, y_prob=y_prob, additional_data=df_test_lemmatized, cls='poniżanie')

#### FN

In [128]:
fn.head(2)

,lemmatized,username,hashtags,ground truth,prediction,probability,error
9,logiko gdzie być sędzia i sąd rppl,darthlexus,[],1,0,0.048288,0.951712
46,jebac pies w sąd wbijać,kebabzlodziej,[],1,0,0.074328,0.925672


In [129]:
most_common_word(fn['lemmatized'], min_count=4)

,word,count
2,sąd,80
125,człowiek,8
46,prokuratura,8
1,sędzia,8
21,mieć,7
177,sada,6
140,głowa,4
278,pinior,4
97,polska,4
11,prawo,4


In [130]:
most_common_phrase(fn['lemmatized'], length=2).head()

,phrase,count
324,sąd prokuratura,4
171,polski sąd,3
48,prokuratura sąd,3
331,ble ble,2
38,cholera sąd,2


In [131]:
most_common_phrase(fn['lemmatized'], length=3).head()

,phrase,count
98,0 pojęcie wymiar,1
516,1989r zbrodniarz pozostać,1
60,2006 roktde zginąć,1
63,23 górnik sąd,1
366,24 duży gów,1


In [132]:
most_common_phrase(fn['lemmatized'], length=4).head()

,phrase,count
86,0 pojęcie wymiar sprawiedliwość,1
446,1989r zbrodniarz pozostać sędzia,1
52,2006 roktde zginąć 23,1
55,23 górnik sąd nierychliwy,1
353,30 rok temu kodziarstwo,1


In [133]:
hashtags(fn)

/home/jakub/Dokumenty/Semestr 9.2/HSD2/src/error_analysis.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['hashtags'] = dff.apply(lambda x: list([ht[1:-1] for ht in x['hashtags'][1:-1].split(', ')]), axis=1)


,hashtag,error
0,#bronka,0.904093
2,#po,0.904093
1,#pinior,0.898449
3,#wtylewizji,0.868241


In [134]:
usernames(fn).head()

,username,error
68,wiihol,0.912762
16,chryzopraz,0.912643
50,pawelpreneta,0.912064
3,aleksanderzgoa,0.911297
42,lobuzica,0.911208


#### FP

In [135]:
fp.head(2)

,lemmatized,username,hashtags,ground truth,prediction,probability,error
4,wątpić brać pod uwaga ogromny latyfundiam mias...,malehammer,[],0,1,0.519291,0.519291
25,to prosty bo sąd być nadal nie oczyścić,bolek_ma_tole,[],0,1,0.620084,0.620084


In [136]:
most_common_word(fp['lemmatized'], min_count=4)

,word,count
18,sąd,35
20,oczyścić,4
12,sada,4
39,sędzia,4


In [137]:
most_common_phrase(fp['lemmatized'], length=2).head()

,phrase,count
18,nadal oczyścić,2
16,prosty sąd,2
17,sąd nadal,2
26,sąd powołać,2
60,sąd prokuratura,2


In [138]:
most_common_phrase(fp['lemmatized'], length=3).head()

,phrase,count
15,prosty sąd nadal,2
16,sąd nadal oczyścić,2
148,130 letni wlascicielcekamienicyot,1
188,140 letni wlasciciel,1
51,2010 roktde 613,1


In [139]:
most_common_phrase(fp['lemmatized'], length=4).head()

,phrase,count
14,prosty sąd nadal oczyścić,2
130,130 letni wlascicielcekamienicyot pluralizm,1
46,2010 roktde 613 tysiąc,1
34,2015 roktde sąd orzec,1
115,2016 wolny sąd 2017,1


In [140]:
hashtags(fp)

/home/jakub/Dokumenty/Semestr 9.2/HSD2/src/error_analysis.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['hashtags'] = dff.apply(lambda x: list([ht[1:-1] for ht in x['hashtags'][1:-1].split(', ')]), axis=1)


,hashtag,error
7,#pomagamlemingom,0.385097
1,#francuskie,0.359912
3,#niewygodnefakty,0.359912
8,#postępowe,0.359912
11,#uśmiechnijsiefrancjo,0.359912
5,#państwowpaństwie,0.359509
9,#sądy,0.307867
2,#mamdość,0.256643
0,#etpc,0.255823
4,#orzeczenie,0.255823


In [141]:
usernames(fp).head()

,username,error
18,iax2hffwvsuf5lg,0.457525
23,najpolski,0.451810
20,kgb08maria,0.439941
24,nova__mova,0.423638
31,zniedziolka,0.409616


#### TN

In [142]:
tn.head(2)

,lemmatized,username,hashtags,ground truth,prediction,probability,error
0,warto poznać śląsk i ślązak zanim zacząć się w...,arozenek,[],0,0,0.298740,0.298740
1,jeszcze nie być całkiem bezkarny sąd jeszcze t...,koziolmk,[],0,0,0.046095,0.046095


In [143]:
most_common_word(tn['lemmatized'], min_count=4)

,word,count
8,sąd,852
39,mieć,120
74,prokuratura,88
160,sada,87
143,polski,54
...,...,...
439,zasada,4
1479,zdanie,4
1567,złodziej,4
1962,łamać,4


In [144]:
most_common_phrase(tn['lemmatized'], length=2).head()

,phrase,count
160,polski sąd,26
643,sąd prokuratura,24
484,prokuratura sąd,18
41,sąd mieć,17
660,niezawisły sąd,16


In [145]:
most_common_phrase(tn['lemmatized'], length=3).head()

,phrase,count
1095,komunizm nadal uważać,6
1096,nadal uważać sąd,6
5349,podporządkować siebie sąd,6
1097,uważać sąd chuligan,6
1094,walczyć komunizm nadal,6


In [146]:
most_common_phrase(tn['lemmatized'], length=4).head()

,phrase,count
940,komunizm nadal uważać sąd,6
941,nadal uważać sąd chuligan,6
939,walczyć komunizm nadal uważać,6
4609,aparat bezpieczeństwo media publiczny,5
851,człowiek platfus móc spać,5


In [147]:
hashtags(tn)

/home/jakub/Dokumenty/Semestr 9.2/HSD2/src/error_analysis.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['hashtags'] = dff.apply(lambda x: list([ht[1:-1] for ht in x['hashtags'][1:-1].split(', ')]), axis=1)


,hashtag,error
51,#państwoprawa,0.228274
93,#zmianypersonalne,0.221594
94,#zwłokaodbioruślubowania,0.221594
15,#dobrazmiana,0.215354
26,#kawaleriaprawna,0.215354
...,...,...
6,#audyty,0.001971
73,#smoleńsk,0.001971
19,#dziejesięhistoria,0.001957
74,#sn,0.001957


In [148]:
usernames(tn).head()

,username,error
17,_zboral,0.249744
205,hmaegerle,0.248289
585,tomaszfuchs1970,0.246309
320,lazacy_lazarz,0.246078
493,przemyslawantas,0.243817


#### TP

In [149]:
tp.head(2)

,lemmatized,username,hashtags,ground truth,prediction,probability,error
55,prokuratura i sąd to nadal stajnia augiasz i o...,trustandcontrol,[],1,1,0.590491,0.409509
60,minąłać sąd do pełny dekomunizacja wraz z dzie...,szlachciura,['#minęła20'],1,1,0.521930,0.478070


In [150]:
most_common_word(tp['lemmatized'], min_count=4)

,word,count
1,sąd,8


In [151]:
most_common_phrase(tp['lemmatized'], length=2).head()

,phrase,count
43,2 domniemanie,1
4,augiasz ogromny,1
48,bezczelnie łamać,1
30,bolszewicki twór,1
9,chroni prezes,1


In [152]:
most_common_phrase(tp['lemmatized'], length=3).head()

,phrase,count
39,2 domniemanie niewinność,1
4,augiasz ogromny pole,1
44,bezczelnie łamać ciekawy,1
28,bolszewicki twór patologia,1
9,chroni prezes sn,1


In [153]:
most_common_phrase(tp['lemmatized'], length=4).head()

,phrase,count
35,2 domniemanie niewinność sąd,1
4,augiasz ogromny pole popis,1
40,bezczelnie łamać ciekawy podstawa,1
26,bolszewicki twór patologia wykorzenić,1
9,chroni prezes sn skandal,1


In [154]:
hashtags(tp)

/home/jakub/Dokumenty/Semestr 9.2/HSD2/src/error_analysis.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['hashtags'] = dff.apply(lambda x: list([ht[1:-1] for ht in x['hashtags'][1:-1].split(', ')]), axis=1)


,hashtag,error
0,#minęła20,0.228551


In [155]:
usernames(tp).head()

,username,error
1,csebastjaniuk,0.212683
3,kuznia1,0.188387
4,szlachciura,0.170690
5,trustandcontrol,0.167697
0,bogdan1409,0.148195


### Stygmatyzacja

In [156]:
fn, fp, tn, tp = split_error_types(y_true=y_test_data, y_pred=y_pred, y_prob=y_prob, additional_data=df_test_lemmatized, cls='stygmatyzacja')

#### FN

In [157]:
fn.head(2)

,lemmatized,username,hashtags,ground truth,prediction,probability,error
10,złodziejstwo członek i sympatyk aws i po powin...,romualdmyga,['#wieszwiecej'],1,0,0.117215,0.882785
18,to jak chcieć on zamknąć sąd wypusccząjać bury...,edytamf,[],1,0,0.325658,0.674342


In [158]:
most_common_word(fn['lemmatized'], min_count=4)

,word,count
5,sąd,144
51,polski,18
92,prokuratura,17
107,sada,17
43,mieć,15
48,prawo,12
126,człowiek,11
56,móc,11
42,polska,10
125,kasta,9


In [159]:
most_common_phrase(fn['lemmatized'], length=2).head()

,phrase,count
96,prokuratura sąd,7
130,sąd kasta,7
153,sąd prokuratura,7
1014,podporządkować siebie,6
51,polski rząd,6


In [160]:
most_common_phrase(fn['lemmatized'], length=3).head()

,phrase,count
927,podporządkować siebie sąd,6
930,aparat bezpieczeństwo media,5
931,bezpieczeństwo media publiczny,5
117,człowiek platfus móc,5
114,dopóki sąd kasta,5


In [161]:
most_common_phrase(fn['lemmatized'], length=4).head()

,phrase,count
802,aparat bezpieczeństwo media publiczny,5
101,człowiek platfus móc spać,5
98,dopóki sąd kasta człowiek,5
795,gazeta szkalywać polski rząd,5
100,kasta człowiek platfus móc,5


In [162]:
hashtags(fn)

/home/jakub/Dokumenty/Semestr 9.2/HSD2/src/error_analysis.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['hashtags'] = dff.apply(lambda x: list([ht[1:-1] for ht in x['hashtags'][1:-1].split(', ')]), axis=1)


,hashtag,error
1,#dyktaturapis,0.904762
6,#studiopolska,0.903122
5,#pinior,0.866877
10,#wymiarniesprawiedliwości,0.866196
7,#wieszwiecej,0.779309
4,#minęła20,0.766969
11,#zmianacałodobowa,0.756686
8,#woronicza17,0.672748
2,#dziejesięnażywo,0.634856
3,#korespondencja,0.634856


In [163]:
usernames(fn).head()

,username,error
110,wavetable20,0.914759
106,urban_arkadiusz,0.913798
23,cezarykrysztopa,0.913771
88,pietro_v_volpi,0.912497
33,donkiszot1,0.911434


#### FP

In [164]:
fp.head(2)

,lemmatized,username,hashtags,ground truth,prediction,probability,error
4,wątpić brać pod uwaga ogromny latyfundiam mias...,malehammer,[],0,1,0.582064,0.582064
11,sąd i etpc oceniać odpzawadliwyskład tk niezba...,andrzejnowak,"['#sądy', '#etpc', '#odpzawadliwyskład', '#tk'...",0,1,0.524196,0.524196


In [165]:
most_common_word(fp['lemmatized'], min_count=4)

,word,count
17,sąd,48
12,sada,9
94,tysiąc,9
96,rok,7
21,tk,7
84,sędzia,6
29,mieć,5
273,wolny,5
35,czas,4
32,człowiek,4


In [166]:
most_common_phrase(fp['lemmatized'], length=2).head()

,phrase,count
16,sąd etpc,3
312,wolny sąd,3
106,2010 roktde,2
108,613 tysiąc,2
104,657 tysiąc,2


In [167]:
most_common_phrase(fp['lemmatized'], length=3).head()

,phrase,count
97,2010 roktde 613,2
95,657 tysiąc 2010,2
90,673 tysiąc rozwód,2
16,etpc oceniać odpzawadliwyskład,2
20,niezbagatelizować zwłokaodbioruvlubowanie wcelu,2


In [168]:
most_common_phrase(fp['lemmatized'], length=4).head()

,phrase,count
88,2010 roktde 613 tysiąc,2
86,657 tysiąc 2010 roktde,2
81,673 tysiąc rozwód rok,2
15,etpc oceniać odpzawadliwyskład tk,2
19,niezbagatelizować zwłokaodbioruvlubowanie wcel...,2


In [169]:
hashtags(fp)

/home/jakub/Dokumenty/Semestr 9.2/HSD2/src/error_analysis.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['hashtags'] = dff.apply(lambda x: list([ht[1:-1] for ht in x['hashtags'][1:-1].split(', ')]), axis=1)


,hashtag,error
11,#pomagamlemingom,0.517937
2,#francuskie,0.488632
5,#niewygodnefakty,0.488632
12,#postępowe,0.488632
18,#uśmiechnijsiefrancjo,0.488632
15,#sądy,0.318535
7,#orzeczenie,0.313645
10,#pokrzywdzony,0.313645
16,#sędziowienazapas,0.313645
21,#śmiałeopinie,0.313645


In [170]:
usernames(fp).head()

,username,error
0,_boreyko_,0.538059
8,bolek_ma_tole,0.519002
14,eric1dale,0.517937
20,holger_d69,0.515471
45,windswaysapp,0.488632


#### TN

In [171]:
tn.head(2)

,lemmatized,username,hashtags,ground truth,prediction,probability,error
0,warto poznać śląsk i ślązak zanim zacząć się w...,arozenek,[],0,0,0.276559,0.276559
1,jeszcze nie być całkiem bezkarny sąd jeszcze t...,koziolmk,[],0,0,0.046981,0.046981


In [172]:
most_common_word(tn['lemmatized'], min_count=4)

,word,count
8,sąd,749
39,mieć,105
57,prokuratura,77
118,sada,69
36,prawo,44
...,...,...
1407,wątpić,4
437,zamknąć,4
372,zasada,4
1341,zdanie,4


In [173]:
most_common_phrase(tn['lemmatized'], length=2).head()

,phrase,count
112,polski sąd,23
546,sąd prokuratura,20
572,niezawisły sąd,17
41,sąd mieć,17
393,prokuratura sąd,14


In [174]:
most_common_phrase(tn['lemmatized'], length=3).head()

,phrase,count
977,komunizm nadal uważać,6
978,nadal uważać sąd,6
979,uważać sąd chuligan,6
976,walczyć komunizm nadal,6
2994,józef pinior trafić,5


In [175]:
most_common_phrase(tn['lemmatized'], length=4).head()

,phrase,count
837,komunizm nadal uważać sąd,6
838,nadal uważać sąd chuligan,6
836,walczyć komunizm nadal uważać,6
2581,józef pinior trafić areszt,5
275,konferencja sada ogród narodowy,4


In [176]:
hashtags(tn)

/home/jakub/Dokumenty/Semestr 9.2/HSD2/src/error_analysis.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['hashtags'] = dff.apply(lambda x: list([ht[1:-1] for ht in x['hashtags'][1:-1].split(', ')]), axis=1)


,hashtag,error
12,#cyrk,0.214080
23,#inpost,0.197547
58,#poznań,0.174922
32,#maziarski,0.140078
16,#duda,0.134086
...,...,...
6,#audyty,0.002021
69,#smoleńsk,0.002021
17,#dziejesięhistoria,0.002003
70,#sn,0.002003


In [177]:
usernames(tn).head()

,username,error
330,marcinnaps,0.249496
327,marcin_kaminski,0.248379
301,lilacliffo,0.247021
167,gkaczmarek,0.246266
283,kueberhan,0.243908


#### TP

In [178]:
tp.head(2)

,lemmatized,username,hashtags,ground truth,prediction,probability,error
38,taki mieć peowski sada czas polikwidować mafia...,kgb08maria,[],1,1,0.759801,0.240199
40,tk musieć być normalny a sąd trzeba wyczyścić ...,wojciechhorwat,[],1,1,0.548427,0.451573


In [179]:
most_common_word(tp['lemmatized'], min_count=4)

,word,count
9,sąd,34
120,działać,4
5,mafia,4
0,mieć,4
33,polska,4
56,prl,4
86,wolny,4


In [180]:
most_common_phrase(tp['lemmatized'], length=2).head()

,phrase,count
92,okrągły stół,2
66,sąd media,2
60,sąd oczyścić,2
86,sąd prokuratura,2
59,sąd sąd,2


In [181]:
most_common_phrase(tp['lemmatized'], length=3).head()

,phrase,count
68,2 domniemanie niewinność,1
134,2016 wolny sąd,1
137,2017 pisowski sędzia,1
114,afera popiół czas,1
117,analogia działać działać,1


In [182]:
most_common_phrase(tp['lemmatized'], length=4).head()

,phrase,count
60,2 domniemanie niewinność sąd,1
117,2016 wolny sąd 2017,1
120,2017 pisowski sędzia pis,1
100,afera popiół czas przedawnienie,1
102,analogia działać działać układ,1


In [183]:
hashtags(tp)

/home/jakub/Dokumenty/Semestr 9.2/HSD2/src/error_analysis.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['hashtags'] = dff.apply(lambda x: list([ht[1:-1] for ht in x['hashtags'][1:-1].split(', ')]), axis=1)


,hashtag,error
3,#woronicza17,0.229825
0,#mamdość,0.192321
1,#minęła20,0.169829
2,#państwowpaństwie,0.090874


In [184]:
usernames(tp).head()

,username,error
17,rokjerzy,0.239588
20,tomiekawakami13,0.228174
14,mac_zetka,0.221505
0,alexja25,0.215215
10,idariusz,0.214253


### Szantaż

In [185]:
fn, fp, tn, tp = split_error_types(y_true=y_test_data, y_pred=y_pred, y_prob=y_prob, additional_data=df_test_lemmatized, cls='szantaż')

#### FN

In [186]:
fn.head(2)

,lemmatized,username,hashtags,ground truth,prediction,probability,error
878,polski sąd móc ja naskoczyć rozumieć jakbym by...,polishheartbeat,[],1,0,0.002278,0.997722


In [187]:
most_common_word(fn['lemmatized'], min_count=4)

,word,count


In [188]:
most_common_phrase(fn['lemmatized'], length=2).head()

,phrase,count
5,jakbym osoba,1
7,medialny kurwa,1
2,móc naskoczyć,1
3,naskoczyć rozumieć,1
6,osoba medialny,1


In [189]:
most_common_phrase(fn['lemmatized'], length=3).head()

,phrase,count
5,jakbym osoba medialny,1
2,móc naskoczyć rozumieć,1
3,naskoczyć rozumieć jakbym,1
6,osoba medialny kurwa,1
0,polski sąd móc,1


In [190]:
most_common_phrase(fn['lemmatized'], length=4).head()

,phrase,count
5,jakbym osoba medialny kurwa,1
2,móc naskoczyć rozumieć jakbym,1
3,naskoczyć rozumieć jakbym osoba,1
0,polski sąd móc naskoczyć,1
4,rozumieć jakbym osoba medialny,1


In [191]:
hashtags(fn)

/home/jakub/Dokumenty/Semestr 9.2/HSD2/src/error_analysis.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['hashtags'] = dff.apply(lambda x: list([ht[1:-1] for ht in x['hashtags'][1:-1].split(', ')]), axis=1)


,hashtag,error


In [192]:
usernames(fn).head()

,username,error
0,polishheartbeat,0.995449


#### FP

In [193]:
fp.head(2)

,lemmatized,username,hashtags,ground truth,prediction,probability,error


In [194]:
most_common_word(fp['lemmatized'], min_count=4)

,word,count


In [195]:
most_common_phrase(fp['lemmatized'], length=2).head()

,phrase,count


In [196]:
most_common_phrase(fp['lemmatized'], length=3).head()

,phrase,count


In [197]:
most_common_phrase(fp['lemmatized'], length=4).head()

,phrase,count


In [198]:
hashtags(fp)

,hashtag,error


In [199]:
usernames(fp).head()

,username,error


#### TN

In [200]:
tn.head(2)

,lemmatized,username,hashtags,ground truth,prediction,probability,error
0,warto poznać śląsk i ślązak zanim zacząć się w...,arozenek,[],0,0,0.005882,0.005882
1,jeszcze nie być całkiem bezkarny sąd jeszcze t...,koziolmk,[],0,0,0.001522,0.001522


In [201]:
most_common_word(tn['lemmatized'], min_count=4)

,word,count
8,sąd,974
55,mieć,129
91,prokuratura,99
37,sada,97
163,polski,57
...,...,...
178,zbrodniarz,4
1656,zdanie,4
2473,zwykły,4
222,łączewski,4


In [202]:
most_common_phrase(tn['lemmatized'], length=2).head()

,phrase,count
715,sąd prokuratura,30
180,polski sąd,28
342,prokuratura sąd,22
57,sąd mieć,18
741,niezawisły sąd,17


In [203]:
most_common_phrase(tn['lemmatized'], length=3).head()

,phrase,count
1258,komunizm nadal uważać,6
1259,nadal uważać sąd,6
6164,podporządkować siebie sąd,6
1260,uważać sąd chuligan,6
1257,walczyć komunizm nadal,6


In [204]:
most_common_phrase(tn['lemmatized'], length=4).head()

,phrase,count
1085,komunizm nadal uważać sąd,6
1086,nadal uważać sąd chuligan,6
1084,walczyć komunizm nadal uważać,6
5323,aparat bezpieczeństwo media publiczny,5
996,człowiek platfus móc spać,5


In [205]:
hashtags(tn)

/home/jakub/Dokumenty/Semestr 9.2/HSD2/src/error_analysis.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['hashtags'] = dff.apply(lambda x: list([ht[1:-1] for ht in x['hashtags'][1:-1].split(', ')]), axis=1)


,hashtag,error
66,#pomagamlemingom,0.000330
24,#francuskie,0.000314
48,#niewygodnefakty,0.000314
68,#postępowe,0.000314
94,#uśmiechnijsiefrancjo,0.000314
...,...,...
25,#gabinetcieni,0.000002
54,#państwopis,0.000002
41,#milewski,0.000002
11,#bronka,0.000002


In [206]:
usernames(tn).head()

,username,error
243,iax2hffwvsuf5lg,0.000456
475,nova__mova,0.000386
714,zniedziolka,0.000367
12,_boreyko_,0.000364
112,bolek_ma_tole,0.000342


#### TP

In [207]:
tp.head(2)

,lemmatized,username,hashtags,ground truth,prediction,probability,error


In [208]:
most_common_word(tp['lemmatized'], min_count=4)

,word,count


In [209]:
most_common_phrase(tp['lemmatized'], length=2).head()

,phrase,count


In [210]:
most_common_phrase(tp['lemmatized'], length=3).head()

,phrase,count


In [211]:
most_common_phrase(tp['lemmatized'], length=4).head()

,phrase,count


In [212]:
hashtags(tp)

,hashtag,error


In [213]:
usernames(tp).head()

,username,error
